In [3]:
import pandas as pd
import pymysql
import os
from datetime import datetime

# CSV 파일 불러오기 및 컬럼명 변경
crawl_date = datetime.today().strftime('%Y-%m-%d')
csv_path = fr"C:\Users\Playdata\Documents\SKN13-1st-3Team\crawling\{crawl_date}.csv"

df = pd.read_csv(csv_path, encoding='utf-8')

df = df.rename(columns={
    '이름': 'station_name',
    '주소': 'address',
    '브랜드': 'brand',
    '지역': 'region',
    '휘발유': 'gasoline_price',
    '경유': 'diesel_price',
    '셀프': 'self_service',
    '세차': 'car_wash',
    '편의점': 'convenience_store',
    '24시간': 'hours_24'
})

# 브랜드명 정리
df['brand'] = df['brand'].apply(lambda x: '알뜰주유소' if '알뜰' in str(x) else x)

brand_name_map = {
    "S-OIL": "S-OIL",
    "SK에너지": "SK에너지",
    "GS칼텍스": "GS칼텍스",
    "현대오일뱅크": "HD현대오일뱅크",
    "HD현대오일뱅크": "HD현대오일뱅크",
    "코끼리주유소": "코끼리주유소",
    "정호주유소": "정호주유소"
}
df['brand'] = df['brand'].map(lambda x: brand_name_map.get(x, x))

df.loc[df['station_name'].str.contains('정호'), 'brand'] = '정호주유소'
df['collected_at'] = crawl_date

# DB 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',  # ← 너의 DB 비밀번호로 변경!
    db='gas_station',  # ← 너의 DB 이름으로 변경!
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

# brand 테이블 입력
brand_list = [
    ["S-OIL", "서울특별시 마포구 백범로 192", "1644-5151"],
    ["SK에너지", "서울특별시 종로구 종로 26 SK빌딩", "02-2121-5114"],
    ["GS칼텍스", "서울특별시 강남구 논현로 508 GS타워", "1544-5151"],
    ["HD현대오일뱅크", "경기도 성남시 분당구 분당수서로477", "1588-5189"],
    ["알뜰주유소","울산광역시 중구 종가로 305","1666-5192"],
    ["코끼리주유소", "서울 은평구 서오릉로 41", "02-384-4000"],
    ["정호주유소", "서울 성동구 고산자로 307", "02-2297-9442"]
]

cursor.execute("DELETE FROM gas_station")  # 기존 데이터 삭제 (필요 시)
cursor.execute("DELETE FROM brand")
conn.commit()

sql_brand = "INSERT INTO brand (brand_name, head_address, cst_service) VALUES (%s, %s, %s)"
cursor.executemany(sql_brand, brand_list)
conn.commit()
 
# brand_id 매핑
cursor.execute("SELECT brand_id, brand_name FROM brand")
brand_rows = cursor.fetchall()
brand_dict = {row['brand_name']: row['brand_id'] for row in brand_rows}
df['brand_id'] = df['brand'].map(brand_dict)

# 쉼표 제거 후 숫자형으로 변환
df['gasoline_price'] = pd.to_numeric(df['gasoline_price'].astype(str).str.replace(',', ''), errors='coerce')
df['diesel_price'] = pd.to_numeric(df['diesel_price'].astype(str).str.replace(',', ''), errors='coerce')

# NaN 값은 0으로 채우고 정수형으로 변환
df = df.fillna({'gasoline_price': 0, 'diesel_price': 0})
df['gasoline_price'] = df['gasoline_price'].astype(int)
df['diesel_price'] = df['diesel_price'].astype(int)

# station_data 리스트 생성
station_data = df[[
    'station_name', 'address', 'region', 'gasoline_price', 'diesel_price',
    'self_service', 'car_wash', 'convenience_store', 'hours_24', 'brand_id'
]].values.tolist()

# INSERT 실행
sql_station = """
    INSERT INTO gas_station (
        station_name, address, region, gasoline_price, diesel_price,
        self_service, car_wash, convenience_store, hours_24, brand_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(sql_station, station_data)
conn.commit()

print("✅ 데이터 삽입 완료!")
conn.close()

ProgrammingError: nan can not be used with MySQL